CNN Script

In [85]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
tf.compat.v1.disable_eager_execution()

tf.__version__

# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128        

In [86]:
from tensorflow.keras.datasets import mnist 
#from tensorflow.examples.tutorials.mnist import input_data
# data = tf.keras.datasets.mnist.load_data()

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Print the sizes of the datasets
print("Size of:")
print("- Training set:\t\t{}".format(len(train_labels)))
print("- Test set:\t\t{}".format(len(test_labels)))

# If you want a validation set, you can create it from the training set:
# For example, let's take the last 5000 images from the training set as a validation set
validation_images, validation_labels = train_images[-5000:], train_labels[-5000:]
train_images, train_labels = train_images[:-5000], train_labels[:-5000]

print("- Validation set:\t{}".format(len(validation_labels)))
print("- Adjusted Training set:\t{}".format(len(train_labels)))


test_cls = np.argmax(test_labels, axis=1) if test_labels.ndim == 2 else test_labels

# Print the first few class labels to verify
print("First few class labels of test set:", test_cls[:10])

# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10

def new_weights(shape):
    return tf.Variable(tf.compat.v1.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer =  tf.compat.v1.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.compat.v1.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

Size of:
- Training set:		60000
- Test set:		10000
- Validation set:	5000
- Adjusted Training set:	55000
First few class labels of test set: [7 2 1 0 4 1 4 9 5 9]


In [87]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer =  tf.compat.v1.nn.relu(layer)

    return layer


In [88]:
x = tf.compat.v1.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.compat.v1.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.compat.v1.argmax(y_true, axis=1)

In [89]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

layer_conv2, weights_conv2 = new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)


layer_flat, num_features = flatten_layer(layer_conv2)


layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)


layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)


y_pred =  tf.compat.v1.nn.softmax(layer_fc2)
y_pred_cls = tf.compat.v1.argmax(y_pred, axis=1)

In [90]:
cross_entropy = tf.compat.v1.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)
cost = tf.compat.v1.reduce_mean(cross_entropy)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.compat.v1.equal(y_pred_cls, y_true_cls)
accuracy = tf.compat.v1.reduce_mean(tf.cast(correct_prediction, tf.float32))


session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())
train_batch_size = 64
# Counter for total number of iterations performed so far.
total_iterations = 0

In [91]:
train_images_flat = train_images.reshape(-1, 784)
def get_batches(data, labels, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size], labels[i:i + batch_size]

# Optimization function
def optimize(num_iterations):
    global total_iterations

    start_time = time.time()

    with tf.compat.v1.Session() as session:
        session.run(tf.compat.v1.global_variables_initializer())

        for i in range(total_iterations, total_iterations + num_iterations):
            # Use flattened images in batches
            for x_batch, y_true_batch in get_batches(train_images_flat, train_labels, train_batch_size):
                # Prepare the feed dictionary for the placeholders
                feed_dict_train = {x: x_batch, y_true: y_true_batch}

                # Run the optimizer using this batch
                session.run(optimizer, feed_dict=feed_dict_train)

                # Print status every 100 iterations
                if i % 100 == 0:
                    acc = session.run(accuracy, feed_dict=feed_dict_train)
                    msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
                    print(msg.format(i + 1, acc))

        # Update the total iterations
        total_iterations += num_iterations

        # Calculate time usage
        end_time = time.time()
        time_dif = end_time - start_time
        print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [92]:
test_batch_size = 256

In [93]:
test_images_flat = test_images.reshape(-1, 784)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

# Define plotting functions
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9

    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(img_shape), cmap='binary')
        xlabel = f"True: {cls_true[i]}"
        if cls_pred is not None:
            xlabel += f", Pred: {cls_pred[i]}"
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

def plot_example_errors(cls_pred, correct):
    incorrect = ~correct
    images = test_images[incorrect]
    cls_pred_incorrect = cls_pred[incorrect]
    cls_true_incorrect = test_cls[incorrect]
    plot_images(images=images[:9], cls_true=cls_true_incorrect[:9], cls_pred=cls_pred_incorrect[:9])

def plot_confusion_matrix(cls_pred):
    cls_true = test_cls
    cm = confusion_matrix(y_true=cls_true, y_pred=cls_pred)
    print(cm)
    plt.matshow(cm)
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

def print_test_accuracy(show_example_errors=False, show_confusion_matrix=False):
    num_test = len(test_images)
    cls_pred = np.zeros(shape=num_test, dtype=int)

    # Session to run the model
    with tf.compat.v1.Session() as session:
        session.run(tf.compat.v1.global_variables_initializer())
        
        i = 0
        while i < num_test:
            j = min(i + test_batch_size, num_test)
            images = test_images_flat[i:j]
            labels = test_labels_one_hot[i:j]
            feed_dict = {x: images, y_true: labels}
            cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
            i = j

        cls_true = test_cls
        correct = (cls_true == cls_pred)
        correct_sum = correct.sum()
        acc = correct_sum / num_test
        print(f"Accuracy on Test-Set: {acc:.1%} ({correct_sum} / {num_test})")

        if show_example_errors:
            print("Example errors:")
            plot_example_errors(cls_pred=cls_pred, correct=correct)

        if show_confusion_matrix:
            print("Confusion Matrix:")
            plot_confusion_matrix(cls_pred=cls_pred)

In [94]:
print_test_accuracy()
optimize(num_iterations=1)
print_test_accuracy()
optimize(num_iterations=99) # We already performed 1 iteration above.
print_test_accuracy(show_example_errors=False)
optimize(num_iterations=900) # We performed 100 iterations above.
print_test_accuracy(show_example_errors=False)
optimize(num_iterations=9000)
print_test_accuracy(show_example_errors=False)
session.close()

Accuracy on Test-Set: 13.7% (1372 / 10000)


ValueError: Cannot feed value of shape (64,) for Tensor y_true_6:0, which has shape (None, 10)